In [1]:
%cd ..


c:\CMI\Applied ML\applied-ML\ASS_2


In [3]:
! dvc init --subdir

Initialized DVC repository.

You can now commit the changes to git.

+---------------------------------------------------------------------+
|                                                                     |
|        DVC has enabled anonymous aggregate usage analytics.         |
|     Read the analytics documentation (and how to opt-out) here:     |
|             <https://dvc.org/doc/user-guide/analytics>              |
|                                                                     |
+---------------------------------------------------------------------+

What's next?
------------
- Check out the documentation: <https://dvc.org/doc>
- Get help and share ideas: <https://dvc.org/chat>
- Star us on GitHub: <https://github.com/iterative/dvc>


## Decouple compute and storage

In [4]:
!dvc remote add -d storage gdrive://13F27u1JrBZY7tL3ChuV3xF6ApOGfe1Gf

Setting 'storage' as a default remote.


In [66]:
%cd ..

c:\CMI\Applied ML\applied-ML\ASS_2


In [67]:
!git add .dvc/config

In [68]:
!git commit -m "Configure Google Drive as DVC remote storage"

[detached HEAD 010f8da] Configure Google Drive as DVC remote storage
 4 files changed, 11 insertions(+), 6 deletions(-)


In [5]:
%cd src

c:\CMI\Applied ML\applied-ML\ASS_2\src


In [6]:
import pandas as pd
from sklearn.model_selection import train_test_split


In [71]:
import re
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize
import nltk

nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\rickc\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\rickc\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\rickc\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [8]:
# Load the raw data
raw_data = pd.read_csv('../data/emails.csv')


In [14]:
# Save the raw data in the 'data' folder
raw_data.to_csv('../data/raw_data.csv', index=False)


In [72]:
def preprocess_data(data):
    # Lowercasing
    data['text'] = data['text'].apply(lambda x: x.lower())
    
    # Removing HTML tags
    data['text'] = data['text'].apply(lambda x: re.sub(r'<.*?>', '', x))
    
    # Removing URLs
    data['text'] = data['text'].apply(lambda x: re.sub(r'http[s]?://\S+', '', x))
    
    # Removing email addresses
    data['text'] = data['text'].apply(lambda x: re.sub(r'\S*@\S*\s?', '', x))
    
    # Removing punctuation and special characters
    data['text'] = data['text'].apply(lambda x: re.sub(r'[^a-z0-9\s]', '', x))
    
    # Tokenization
    data['text'] = data['text'].apply(lambda x: word_tokenize(x))
    
    # Removing stopwords
    stop_words = set(stopwords.words('english'))
    data['text'] = data['text'].apply(lambda x: [word for word in x if word not in stop_words])
    
    # Lemmatization
    lemmatizer = WordNetLemmatizer()
    data['text'] = data['text'].apply(lambda x: [lemmatizer.lemmatize(word) for word in x])
    
    # Joining tokens back to string
    data['text'] = data['text'].apply(lambda x: ' '.join(x))
    
    return data

In [73]:
preprocessed_raw_data = preprocess_data(raw_data)

In [74]:
# Split the data into train/validation/test with the specified random seeds
train, test = train_test_split(preprocessed_raw_data, test_size=0.2, random_state=42)
train, val = train_test_split(train, test_size=0.25, random_state=42) # 0.25 x 0.8 = 0.2


In [75]:
# Save the splits in the 'data' folder
train.to_csv('../data/train.csv', index=False)
val.to_csv('../data/validation.csv', index=False)
test.to_csv('../data/test.csv', index=False)

In [76]:
%cd ../data

c:\CMI\Applied ML\applied-ML\ASS_2\data


In [77]:
!dvc add raw_data.csv train.csv validation.csv test.csv

⠋ Checking graph



In [78]:
!git add raw_data.csv.dvc train.csv.dvc validation.csv.dvc test.csv.dvc .gitignore

In [79]:
!git commit -m "Track data with DVC"

[detached HEAD 88a48ba] Track data with DVC
 3 files changed, 6 insertions(+), 6 deletions(-)


In [80]:
!dvc config core.autostage true

In [81]:
!dvc push

3 files pushed


In [82]:
%cd ../src

c:\CMI\Applied ML\applied-ML\ASS_2\src


In [83]:
# Update the splits with a different random seed
train, test = train_test_split(preprocessed_raw_data, test_size=0.2, random_state=24)
train, val = train_test_split(train, test_size=0.25, random_state=24)


In [84]:
# Save the updated splits
train.to_csv('../data/train.csv', index=False)
val.to_csv('../data/validation.csv', index=False)
test.to_csv('../data/test.csv', index=False)


In [85]:
%cd ../data

c:\CMI\Applied ML\applied-ML\ASS_2\data


In [86]:
# Add the updated datasets to DVC
!dvc add train.csv validation.csv test.csv

⠋ Checking graph



In [87]:
!git add train.csv.dvc validation.csv.dvc test.csv.dvc

In [88]:
!git commit -m "Update data splits with new random seed"

[detached HEAD 3bbe9d7] Update data splits with new random seed
 3 files changed, 6 insertions(+), 6 deletions(-)


In [89]:
!dvc push

3 files pushed


In [90]:
!git log

commit 3bbe9d7ec105b3f38e1c60cc4f978e0bc41eb204
Author: shreyanc07 <rickchakraborty90@gmail.com>
Date:   Sun Feb 18 17:04:13 2024 +0530

    Update data splits with new random seed

commit 88a48ba257b93a4a053153a701e076095bebc718
Author: shreyanc07 <rickchakraborty90@gmail.com>
Date:   Sun Feb 18 17:03:21 2024 +0530

    Track data with DVC

commit 010f8daf5d96a214f6d61f5431cb4e73202026dc
Author: shreyanc07 <rickchakraborty90@gmail.com>
Date:   Sun Feb 18 16:46:35 2024 +0530

    Configure Google Drive as DVC remote storage

commit 972a876a4ea61fd7aa483d8cc63bea3980e4ab63
Author: shreyanc07 <rickchakraborty90@gmail.com>
Date:   Sun Feb 18 16:14:25 2024 +0530

    Track data with DVC

commit 26dbfbf5eca45f4f889faf03d3c6436bb9daea23
Author: Shreyan Chakraborty <113718043+shreyanc07@users.noreply.github.com>
Date:   Sun Feb 18 15:32:02 2024 +0530

    Delete ASS_2 directory

commit 3cf437f79ac8b79571ef76165952fd662b9df5e5
Author: shreyanc07 <rickchakraborty90@gmail.com>
Date:   Sun Feb 18

### <first_version_hash> = 88a48ba257b93a4a053153a701e076095bebc718

In [91]:
%cd ../data

c:\CMI\Applied ML\applied-ML\ASS_2\data


In [92]:
!git checkout 88a48ba257b93a4a053153a701e076095bebc718 train.csv.dvc validation.csv.dvc test.csv.dvc


Updated 3 paths from d7763da


In [93]:
!dvc checkout

M       validation.csv
M       test.csv
M       train.csv


## Printing Distribution of the original data split

In [94]:
# Print the distribution of the original split
for split in ['train', 'validation', 'test']:
    df = pd.read_csv(f'../data/{split}.csv')
    distribution = df['spam'].value_counts()
    print(f"Distribution in {split}.csv: \n{distribution}\n")

Distribution in train.csv: 
spam
0    2619
1     817
Name: count, dtype: int64

Distribution in validation.csv: 
spam
0    885
1    261
Name: count, dtype: int64

Distribution in test.csv: 
spam
0    856
1    290
Name: count, dtype: int64



### <updated_version_hash> = 3bbe9d7ec105b3f38e1c60cc4f978e0bc41eb204

In [95]:
!git checkout 3bbe9d7ec105b3f38e1c60cc4f978e0bc41eb204 train.csv.dvc validation.csv.dvc test.csv.dvc

Updated 3 paths from 784077b


In [96]:
!dvc checkout

M       test.csv
M       train.csv
M       validation.csv


## Printing the distribution of the Updated data split

In [97]:
# Print the distribution of the updated spilt
for split in ['train', 'validation', 'test']:
    df = pd.read_csv(f'../data/{split}.csv')
    distribution = df['spam'].value_counts()
    print(f"Distribution in {split}.csv: \n{distribution}\n")

Distribution in train.csv: 
spam
0    2606
1     830
Name: count, dtype: int64

Distribution in validation.csv: 
spam
0    881
1    265
Name: count, dtype: int64

Distribution in test.csv: 
spam
0    873
1    273
Name: count, dtype: int64



In [98]:
%ls

 Volume in drive C is OS
 Volume Serial Number is 38A6-AE1E

 Directory of c:\CMI\Applied ML\applied-ML\ASS_2\data

02/18/2024  05:06 PM    <DIR>          .
02/18/2024  04:08 PM    <DIR>          ..
02/18/2024  04:12 PM                55 .gitignore
02/04/2024  01:03 PM         8,954,755 emails.csv
02/18/2024  05:03 PM         8,959,672 raw_data.csv
02/18/2024  05:03 PM               100 raw_data.csv.dvc
02/18/2024  05:06 PM         1,153,422 test.csv
02/18/2024  05:06 PM                96 test.csv.dvc
02/18/2024  05:06 PM         3,471,277 train.csv
02/18/2024  05:06 PM                97 train.csv.dvc
02/18/2024  05:06 PM         1,197,751 validation.csv
02/18/2024  05:06 PM               102 validation.csv.dvc
              10 File(s)     23,737,327 bytes
               2 Dir(s)  74,723,930,112 bytes free


In [99]:
%cd ../src

c:\CMI\Applied ML\applied-ML\ASS_2\src
